In [1]:
# general imports
import pathlib
import os
import cobra
import tqdm

In [2]:
current_dir = pathlib.Path(os.getcwd())
model_dir = current_dir.parent / "models"

In [3]:
# load Human-GEM annotations
rxn_annotations = {}
gene_annotations = {}
with cobra.io.read_sbml_model(model_dir / "Human-GEM.xml") as ihuman:
    for rxn in ihuman.reactions:
        rxn_annotations[rxn.id] = ihuman.reactions.get_by_id(rxn.id).annotation
    for gene in ihuman.genes:
        gene_annotations[gene.id] = ihuman.genes.get_by_id(gene.id).annotation

In [5]:
# iterate over files in the modles folder and check for files that have not been translated to xml yet
for model_file in model_dir.iterdir():
    if model_file.name.endswith(".mat"):
        xml_name = model_file.name.replace(".mat", ".xml")
        if not (model_dir / xml_name).is_file():
            print(f"Currently re-annotating: {model_file.name}")
            print("*loading model*")
            with cobra.io.load_matlab_model(model_file) as model:
                # iterating over reactions in the model and assigning the respective annotations lost in ftINIT
                print("*updating reaction annotations*")
                for reaction in tqdm(model.reactions):
                    model.reactions.get_by_id(reaction.id).annotation.update(rxn_annotations(reaction.id))
                print("*updating gene annotations*")
                for gene in tqdm(model.genes):
                    model.genes.get_by_id(gene.id).annotation.update(gene_annotations(reaction.id))
                print("*saving model*")
                cobra.io.write_sbml_model(model, model_dir / xml_name)
            print(f"Created model: {xml_name}\n")

Currently re-annotating : skin_no_sun_ftINIT_1+1_model.mat
Created model: skin_no_sun_ftINIT_1+1_model.xml
Currently re-annotating : skin_no_sun_ftINIT_model.mat
Created model: skin_no_sun_ftINIT_model.xml
Currently re-annotating : skin_sun_ftINIT_1+1_model.mat


: 